In [1]:
import numpy as np
import json
import matplotlib.pyplot as plt
from time import time
import os

dir = os.getcwd()
datadir = os.path.join(dir, 'vessel_data/CSAD')
filepath = os.path.join(datadir, [file for file in os.listdir(datadir) if file.endswith('.json')][0])

In [2]:
def timeit(func):
    # Function for calculating execution time.
    def wrapper(*args, **kwargs):
        start = time()
        result = func(*args, **kwargs)
        end = time()
        print(f"Execution time of '{func.__name__}': {end - start}")
        return result
    return wrapper

In [3]:
# Get the parameters from CSAD

with open(filepath, 'r') as f:
    data = json.load(f)



Mrb = np.asarray(data['MRB'])
A = np.asarray(data['A'])
B = np.asarray(data['B'])
C = np.asarray(data['C'])
freqs = np.asarray(data['freqs'])
headings = np.asarray(data['headings'])
Bv = np.asarray(data['Bv'])
# Store RAOs as complex values -> magnitude as real value and phase as imaginary
forceRAO = np.asarray(data['forceRAO']['amp'])[:, :, :, 0] + np.asarray(data['forceRAO']['phase'])[:, :, :, 0]*1j
motionRAO = np.asarray(data['motionRAO']['amp'])[:, :, :, 0] + np.asarray(data['motionRAO']['phase'])[:, :, :, 0]*1j
QTFs = np.asarray(data['driftfrc']['amp'])[:, :, :, 0]
QTFs[0].shape

(36, 36)

In [4]:
from numpy import dtype


@timeit
def full_qtf(qtfs, headings, freqs, wave_freqs, dof=0):
    # Generate full QTF by Newman approximation
    num_headings = len(headings)
    num_freq = len(wave_freqs)
    freq_indices = [np.argmin(np.abs(freqs - w)) for w in wave_freqs]
    Q = np.zeros((num_headings, num_freq, num_freq))
    Qjk = qtfs[dof].copy()
    for i in range(num_headings):
        for j, freq_j in enumerate(freq_indices):
            Q_j = Qjk[freq_j, i]
            for k, freq_k in enumerate(freq_indices):
                Q_k = Qjk[freq_k, i]
                Q[i, j, k] = 0.5*(Q_j + Q_k)
    return Q

@timeit
def full_qtf_alt(qtfs, headings, freqs, wave_freqs, dof=0):
    num_headings = len(headings)
    num_freq = len(wave_freqs)
    freq_indices = [np.argmin(np.abs(freqs - w)) for w in wave_freqs]
    Q = np.zeros((num_headings, num_freq, num_freq))
    Qdiag = qtfs[dof, [freq_indices], :].copy()
    for i in range(len(headings)):
        Q[i] = 0.5*(Qdiag[0, :, i, None] + Qdiag[0, :, i])
    return Q

N = 1000 # Number of wave frequencies
wave_freqs = np.linspace(0.1, 3.14, N, endpoint=False)
# wave_freqs = freqs

print(f"Result diff: {np.sum(full_qtf_alt(QTFs, headings, freqs, wave_freqs, dof=0) - full_qtf(QTFs, headings, freqs, wave_freqs, dof=0))}")

@timeit
def full_qtf_6dof():
    Qd_surge = full_qtf(QTFs, headings, freqs, wave_freqs, dof=0)
    Qd_sway = full_qtf(QTFs, headings, freqs, wave_freqs, dof=1)
    Qd_heave = full_qtf(QTFs, headings, freqs, wave_freqs, dof=2)
    Qd_roll = full_qtf(QTFs, headings, freqs, wave_freqs, dof=3)
    Qd_pitch = full_qtf(QTFs, headings, freqs, wave_freqs, dof=4)
    Qd_yaw = full_qtf(QTFs, headings, freqs, wave_freqs, dof=5)
    return np.vstack([[Qd_surge], [Qd_sway], [Qd_heave], [Qd_roll], [Qd_pitch], [Qd_yaw]])

Qd = full_qtf_6dof()

Execution time of 'full_qtf_alt': 0.5728392601013184
Execution time of 'full_qtf': 29.409154653549194
Result diff: 0.0
Execution time of 'full_qtf': 28.420010328292847
Execution time of 'full_qtf': 23.67535972595215
Execution time of 'full_qtf': 24.154837608337402
Execution time of 'full_qtf': 20.330845832824707
Execution time of 'full_qtf': 20.82188582420349
Execution time of 'full_qtf': 20.962272882461548
Execution time of 'full_qtf_6dof': 147.39528608322144


In [5]:
np.diagonal(Qd[0, 0])

array([ 3.17367367e-01,  3.17367367e-01,  3.17367367e-01,  3.17367367e-01,
        3.17367367e-01,  3.17367367e-01,  3.17367367e-01,  3.17367367e-01,
        3.17367367e-01,  3.17367367e-01,  3.17367367e-01,  3.17367367e-01,
        3.17367367e-01,  3.17367367e-01,  3.17367367e-01,  3.17367367e-01,
        3.17367367e-01,  3.17367367e-01,  3.17367367e-01,  5.94219942e+00,
        5.94219942e+00,  5.94219942e+00,  5.94219942e+00,  5.94219942e+00,
        5.94219942e+00,  5.94219942e+00,  5.94219942e+00,  5.94219942e+00,
        5.94219942e+00,  5.94219942e+00,  5.94219942e+00,  5.94219942e+00,
        5.94219942e+00,  5.94219942e+00,  5.94219942e+00,  5.94219942e+00,
        5.94219942e+00,  5.94219942e+00,  5.94219942e+00,  5.94219942e+00,
        5.94219942e+00,  5.94219942e+00,  5.94219942e+00,  1.38092320e+01,
        1.38092320e+01,  1.38092320e+01,  1.38092320e+01,  1.38092320e+01,
        1.38092320e+01,  1.38092320e+01,  1.38092320e+01,  1.38092320e+01,
        1.38092320e+01,  

In [6]:
QTFs[0].shape

(36, 36)

In [7]:
Qd_surge_heading0_diag = QTFs[0, :, 0]

In [8]:
Qd_surge_heading0 = 0.5*(Qd_surge_heading0_diag[:, None] + Qd_surge_heading0_diag)

In [9]:
freqs

array([0.1047198, 0.2094395, 0.2513274, 0.3141593, 0.330694 , 0.3490659,
       0.3695991, 0.3926991, 0.4053668, 0.418879 , 0.4333231, 0.448799 ,
       0.4654211, 0.483322 , 0.5026549, 0.5235988, 0.5463639, 0.5711987,
       0.5983986, 0.6283185, 0.6613879, 0.6981317, 0.7391983, 0.7853982,
       0.8377581, 0.8975979, 0.9666439, 1.047198 , 1.142397 , 1.256637 ,
       1.396263 , 1.570796 , 1.795196 , 2.094395 , 2.513274 , 3.141593 ])

In [10]:
from wave_loads import WaveLoad

In [11]:
amps = np.linspace(0, 3, N)
phase = np.random.random(N)*2*np.pi



waveload = WaveLoad(amps, wave_freqs, phase, headings, 0)
waveload._full_qtf_6dof(headings, freqs, QTFs)

Execution time of _full_qtf_6dof: 3.0354


array([[[[ 3.17367367e-01,  3.17367367e-01,  3.17367367e-01, ...,
           3.94089646e+04,  3.94089646e+04,  3.94089646e+04],
         [ 3.17367367e-01,  3.17367367e-01,  3.17367367e-01, ...,
           3.94089646e+04,  3.94089646e+04,  3.94089646e+04],
         [ 3.17367367e-01,  3.17367367e-01,  3.17367367e-01, ...,
           3.94089646e+04,  3.94089646e+04,  3.94089646e+04],
         ...,
         [ 3.94089646e+04,  3.94089646e+04,  3.94089646e+04, ...,
           7.88176118e+04,  7.88176118e+04,  7.88176118e+04],
         [ 3.94089646e+04,  3.94089646e+04,  3.94089646e+04, ...,
           7.88176118e+04,  7.88176118e+04,  7.88176118e+04],
         [ 3.94089646e+04,  3.94089646e+04,  3.94089646e+04, ...,
           7.88176118e+04,  7.88176118e+04,  7.88176118e+04]],

        [[ 3.25684763e-01,  3.25684763e-01,  3.25684763e-01, ...,
           3.84563115e+04,  3.84563115e+04,  3.84563115e+04],
         [ 3.25684763e-01,  3.25684763e-01,  3.25684763e-01, ...,
           3.84563115e

In [12]:
# Implempentation in wave loads module corresponds to that done in this notebook
# However - the speed is much faster in the wave module function.
# about 50 times faster actually.
np.sum(full_qtf_6dof() - waveload._full_qtf_6dof(headings, freqs, QTFs))

Execution time of 'full_qtf': 28.307363510131836
Execution time of 'full_qtf': 28.645212411880493
Execution time of 'full_qtf': 28.052868604660034
Execution time of 'full_qtf': 22.938809871673584
Execution time of 'full_qtf': 24.00285792350769
Execution time of 'full_qtf': 24.917287588119507
Execution time of 'full_qtf_6dof': 165.3239040374756
Execution time of _full_qtf_6dof: 2.5802


0.0

In [14]:
165.32/2.58

64.07751937984496